### **<font color='blue'> Artistic Colorizer </font>**

#◢ DeOldify - Colorize your own photos!

####**Credits:**

Special thanks to:

Matt Robinson and María Benavente for pioneering the DeOldify image colab notebook.  

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU


# Connect to Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#◢ Git clone and install DeOldify

In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2609, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 2609 (delta 87), reused 209 (delta 70), pack-reused 2346
Receiving objects: 100% (2609/2609), 69.71 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (1170/1170), done.


In [ ]:
cd DeOldify

/content/DeOldify


#◢ Setup

In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

In [ ]:
!pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.3 MB/s eta 0:00:00
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654001 sha256=9665f129d6c607558fa2dafb24fd4ed6988f66837ffc97ec4913a9700cec0649
  Stored in directory: /root/.cach

In [ ]:
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.


In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

mkdir: cannot create directory ‘models’: File exists
--2024-01-03 18:40:20--  https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.199.36.10
Connecting to data.deepai.org (data.deepai.org)|138.199.36.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255144681 (243M) [application/octet-stream]
Saving to: ‘./models/ColorizeArtistic_gen.pth’

./models/ColorizeAr 100%[===================>] 243.32M  4.99MB/s    in 50s     

2024-01-03 18:41:10 (4.88 MB/s) - ‘./models/ColorizeArtistic_gen.pth’ saved [255144681/255144681]



In [ ]:
colorizer = get_image_colorizer(artistic=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can als

#◢ Instructions

### source_url
Type in a url to a direct link of an image.  Usually that means they'll end in .png, .jpg, etc. NOTE: If you want to use your own image, upload it first to a site like Imgur.

### render_factor
The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality images in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality images, but the colors may get slightly washed out.

### watermarked
Selected by default, this places a watermark icon of a palette at the bottom left corner of the image.  This is intended to be a standard way to convey to others viewing the image that it is colorized by AI. We want to help promote this as a standard, especially as the technology continues to improve and the distinction between real and fake becomes harder to discern. This palette watermark practice was initiated and lead by the company MyHeritage in the MyHeritage In Color feature (which uses a newer version of DeOldify than what you're using here).

#### How to Download a Copy
Simply right click on the displayed image and click "Save image as..."!

## Pro Tips

You can evaluate how well the image is rendered at each render_factor by using the code at the bottom (that cell under "See how well render_factor values perform on a frame here").

#◢ Colorize!!

In [ ]:
%%time

# path to JSONL file which lists the source photo URLs to colorize
#photo_urls = '/content/gdrive/My Drive/SigneBrander/finna-records-signe.jsonl'
#output_dir = '/content/gdrive/My Drive/SigneBrander/finna-colorized-signe'
photo_urls = '/content/gdrive/My Drive/SigneBrander/finna-records-timiri.jsonl'
output_dir = '/content/gdrive/My Drive/SigneBrander/finna-colorized-timiri'
# base URL for images
BASE_URL = 'https://finna.fi'

RENDER_FACTOR=40

import json
import io

def fetch_image(image_url):
  response = requests.get(image_url, stream=True)
  imgdata = io.BytesIO(response.content)
  return PIL.Image.open(imgdata).convert('RGB')

def colorize(orig, render_factor=35):
  origpath = 'orig.png'
  orig.save(origpath)
  colorized = colorizer.get_transformed_image(path=origpath, render_factor=render_factor)
  return colorized


with open(photo_urls) as photo_urls_file:
  for line in photo_urls_file:
    record = json.loads(line)
    record_id = record['id']
    try:
      image_url = BASE_URL + record['images'][0]
    except IndexError:
      continue
    print(record_id, image_url)
    orig = fetch_image(image_url)
    imgpath = f"{output_dir}/{record_id}.jpg"
    if os.path.exists(imgpath):
      print(f"image {record_id} already found")
      continue
    print(f"colorizing {record_id}")
    colorized = colorize(orig, RENDER_FACTOR)
    colorized.save(imgpath)
    #break


museovirasto.DD6ACC4BDD65A3CE7C6CDE32BE0DC0C3 https://finna.fi/Cover/Show?source=Solr&id=museovirasto.DD6ACC4BDD65A3CE7C6CDE32BE0DC0C3&index=0&size=large
colorizing museovirasto.DD6ACC4BDD65A3CE7C6CDE32BE0DC0C3
museovirasto.7DF931CBE18A0B518D9940AA77A021CC https://finna.fi/Cover/Show?source=Solr&id=museovirasto.7DF931CBE18A0B518D9940AA77A021CC&index=0&size=large
colorizing museovirasto.7DF931CBE18A0B518D9940AA77A021CC
museovirasto.7D6C947B5104612A9E030682F8C77460 https://finna.fi/Cover/Show?source=Solr&id=museovirasto.7D6C947B5104612A9E030682F8C77460&index=0&size=large
colorizing museovirasto.7D6C947B5104612A9E030682F8C77460
museovirasto.68965205F6A5673CF44DDEFBB7B54EAE https://finna.fi/Cover/Show?source=Solr&id=museovirasto.68965205F6A5673CF44DDEFBB7B54EAE&index=0&size=large
colorizing museovirasto.68965205F6A5673CF44DDEFBB7B54EAE
museovirasto.E9A7C2E7765093EC5C3E05F3619854CF https://finna.fi/Cover/Show?source=Solr&id=museovirasto.E9A7C2E7765093EC5C3E05F3619854CF&index=0&size=large
co

In [ ]:
source_url = 'https://finna.fi/Cover/Show?source=Solr&id=hkm.0369DDC7-2051-44EB-B0D6-370B2DBD80A0&index=0&size=large' #@param {type:"string"}
render_factor = 32  #@param {type: "slider", min: 7, max: 40}
watermarked = False #@param {type:"boolean"}

if source_url is not None and source_url !='':
    image_path = colorizer.plot_transformed_image_from_url(url=source_url, render_factor=render_factor, compare=True, watermarked=watermarked)
    show_image_in_notebook(image_path)
else:
    print('Provide an image url and try again.')

## See how well render_factor values perform on the image here

In [ ]:
for i in range(10,42,2):
    colorizer.plot_transformed_image('test_images/image.png', render_factor=i, display_render_factor=True, figsize=(8,8))

---
#⚙ Recommended image sources
* [/r/TheWayWeWere](https://www.reddit.com/r/TheWayWeWere/)